In [11]:
import pandas as pd
import os 
import re

In [20]:
path = "/Users/joel/Desktop/SUTD /T5/SDS/MY OWN DATA/combined products done.xlsx"


In [21]:


data = pd.read_excel(path)
print(data.head())
print(data.info())


         RecordedDate     Products Categories Feedback Categories  \
0 2024-04-05 09:21:00  Investments    Vickers         Application   
1 2024-04-05 09:23:00  Investments    Vickers     Process Related   
2 2024-04-05 09:24:00  Investments     Others              Others   
3 2024-04-05 09:26:00  Investments     Others              Others   
4 2024-04-05 09:27:00  Investments    Vickers         Application   

                                                 Q2b  Q6a  Q6b  Q10  Q19  
0         The app is easy to use and straightforward  NaN  NaN  NaN  NaN  
1                                   All info linked.  NaN  NaN  NaN  NaN  
2                                                NIL  NaN  NaN  NaN  NaN  
3                                                 na  NaN  NaN  NaN  NaN  
4  I was completely new to this but application w...  NaN  NaN  NaN  NaN  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 604 entries, 0 to 603
Data columns (total 9 columns):
 #   Column               Non-N

In [14]:
import nltk

# Download necessary NLTK resources
nltk.download('punkt')  # For tokenization
nltk.download('wordnet')  # For lemmatization
nltk.download('stopwords')  # For stopwords
nltk.download('omw-1.4')


[nltk_data] Downloading package punkt to /Users/joel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/joel/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/joel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/joel/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')

feedback_texts = data['Q2b'][2:] 


stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess(text):
    if not isinstance(text, str):
        return ""
    words = nltk.word_tokenize(text.lower())
    filtered_words = [lemmatizer.lemmatize(word) for word in words if word.isalnum() and not word in stop_words]
    return ' '.join(filtered_words)
print(feedback_texts)
processed_texts = feedback_texts.apply(preprocess)


tfidf_vectorizer = TfidfVectorizer(max_features=1000) 
tfidf_matrix = tfidf_vectorizer.fit_transform(processed_texts)


kmeans = KMeans(n_clusters=6, random_state=42) 
kmeans.fit(tfidf_matrix)


data['cluster'] = [None]*2 + list(kmeans.labels_)  

print(data[['Q2b', 'cluster']])


[nltk_data] Downloading package stopwords to /Users/joel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/joel/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


2                                                    NIL
3                                                     na
4      I was completely new to this but application w...
5               it didnt take forever to open an account
6      Everything could be done from my phone and the...
                             ...                        
599    I think as a bit early for our appointment, sh...
600    Listen to my needs and recommending the best p...
601    We only started to work with Chanel recently s...
602                                             Punctual
603                   These are important values to me. 
Name: Q2b, Length: 602, dtype: object
                                                   Q2b  cluster
0           The app is easy to use and straightforward      NaN
1                                     All info linked.      NaN
2                                                  NIL      5.0
3                                                   na      4.0
4    I was comp

In [29]:
# Print out texts in each cluster to manually review them
for i in range(5):  # Adjust based on the number of clusters you have
    print(f"\nCluster {i} texts:")
    print(data.loc[data['cluster'] == i, 'Q2b'].tolist()[:10])  # Print first 10 entries for brevity

# Assuming you've examined the clusters, you can now assign labels
# cluster_labels = {
#     0: '',
#     1: '',
#     2: '',
#     3: '',
#     4: ''
# }

# # Map the labels to your data
# data['cluster_label'] = data['cluster'].map(cluster_labels)
# print(data[['Q2b', 'cluster_label']].head(10))



Cluster 0 texts:
['Can just apply via ibanking. No additional information need to provide', 'Very responsive to queries and clarification. Helpful in clarifying my financial planning doubts\xa0', 'I am able to reach my RM easily by email or whatsapp', 'Bella has provided exceptional service and is always extremely prompt whenever I needed her assistance.\xa0', "Bella is a very personalble RM that I've ever come across; rare gem! Natural affable quality & not motivated by bank's 'target'", "She's also easily accessible including outside her working hours; a very industrious RM's trait/s too.", 'Bella respond to my calls/messages in timely fashion.\xa0', 'Bella follows up on my requests in timely and effective manner.\xa0', 'My RM is very responsive whenever I need her help\xa0', 'RM is just a call away to answer my queries.\xa0']

Cluster 1 texts:
['RM followed up with an email of the prospectors of the products I requested for, but he sent me the prospectors of the wrong product. He o

Cluster 2 : application process, app good 